In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

# import sys
# sys.path.append("/home/z5297792/UNSW-MRes/MRes/modules") 
# from utils import 

df_eddies = pd.read_pickle(f'/srv/scratch/z5297792/Chapter2/SEACOFS_26yr_Eddy_Dataset/df_eddies_processed_1462_10650.pkl')
df_eddies


,Eddy,Day,Cyc,Lon,Lat,ic,jc,xc,yc,w,Q11,Q12,Q22,Rc,psi0,Age,Date,fname
0,1,1462,CE,156.969773,-28.461100,179,274,505.897600,1353.012251,-0.000033,-0.008769,0.001546,-0.007521,69.500036,70.832145,57,1994-01-02,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
1,1,1463,CE,156.993096,-28.433992,179,275,507.149717,1356.633687,-0.000030,-0.007934,0.000890,-0.007072,52.250752,156.388574,57,1994-01-03,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
2,1,1464,CE,157.097940,-28.381485,181,276,515.130000,1365.650728,-0.000028,-0.006920,0.001050,-0.006852,67.000485,56.418837,57,1994-01-04,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
3,1,1465,CE,157.094334,-28.315658,181,278,512.570772,1372.426342,-0.000033,-0.006959,0.002144,-0.009312,52.001786,31.355771,57,1994-01-05,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
4,1,1466,CE,157.106324,-28.346479,181,277,514.728351,1369.599726,-0.000029,-0.005883,0.001179,-0.008429,56.251116,29.524244,57,1994-01-06,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115993,2927,10646,AE,157.825620,-37.030656,248,97,850.502045,476.447416,0.000013,0.002921,0.000642,0.003759,72.751791,-76.302293,24,2019-02-24,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
115994,2927,10647,AE,157.880796,-36.972725,249,98,853.608024,484.316677,0.000014,0.002819,0.000983,0.003948,74.250924,-47.064730,24,2019-02-25,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
115995,2927,10648,AE,157.868516,-36.874363,248,100,849.759258,494.400614,0.000014,0.002968,0.000566,0.003910,75.750780,-79.414689,24,2019-02-26,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
115996,2927,10649,AE,157.753058,-36.882303,246,99,840.037603,489.981707,0.000013,0.002804,0.000722,0.003863,76.000006,-92.084211,24,2019-02-27,/srv/scratch/z3533156/26year_BRAN2020/outer_av...


In [15]:
fnumbers = []
for day in df_eddies.Day.unique():
    fnumbers.append(1461 + ((day - 1462) // 30)*30)
fnumbers = np.unique(fnumbers)

u_mean = None
v_mean = None
total_days = len(df_eddies.Day.unique())
for f, fnumber in enumerate(fnumbers):
    u_raw = np.load(f'/srv/scratch/z5297792/Climatology/u_v/u_{fnumber:05d}.npy')
    v_raw = np.load(f'/srv/scratch/z5297792/Climatology/u_v/v_{fnumber:05d}.npy')
    u_raw = np.where(np.abs(u_raw) > 1e30, np.nan, u_raw).astype(float)
    v_raw = np.where(np.abs(v_raw) > 1e30, np.nan, v_raw).astype(float)

    local_days = u_raw.shape[-1]
    s = local_days / total_days
    if f == 0:
        u_mean, v_mean = s * np.mean(u_raw, axis=3), s * np.mean(v_raw, axis=3)
    else:
        u_mean += s * np.mean(u_raw, axis=3)
        v_mean += s * np.mean(v_raw, axis=3)
        
    if f % 10 == 0:
        print(f)



0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300


In [17]:
dic_mean_flow = {'u_mean': u_mean, 'v_mean': v_mean}
with open(f"/srv/scratch/z5297792/Climatology/dic_mean_flow.pkl", "wb") as f:
    pickle.dump(dic_mean_flow, f)
        

In [ ]:
break

In [ ]:
df_AE = df_eddies[df_eddies.Cyc=='AE']
df_CE = df_eddies[df_eddies.Cyc=='CE']

In [ ]:
df_eddies.Age.mean()

In [ ]:
df_AE.Age.mean()

In [ ]:
df_CE.Age.mean()

In [ ]:
len(df[df.Cyc=='CE'])

In [ ]:
df_eddies_OLD = pd.read_pickle('/srv/scratch/z5297792/Clim_data/df_eddies_processed_OLD.pkl')
sample_eddies = [8, 896, 2504, 2749, 382, 926, 1394, 1967]
df_sample = df_eddies_OLD[df_eddies_OLD.Eddy.isin(sample_eddies)].copy()
df_sample


In [ ]:
def rmse_with_padding(a, b):
    a = np.asarray(a, dtype=float)
    b = np.asarray(b, dtype=float)
    n = max(a.size, b.size)
    a = np.pad(a, (0, n - a.size), constant_values=np.nan)
    b = np.pad(b, (0, n - b.size), constant_values=np.nan)
    return np.sqrt(np.nanmean((a - b) ** 2))

eddy = 8
df_eddy = df_sample[df_sample.Eddy == eddy]

rmse_min = np.inf
eddy_match = None
for e, df in df_eddies.groupby('Eddy'):
    if df.empty or df.iloc[0].Cyc != df_eddy.iloc[0].Cyc:
        continue
    rmse = np.hypot(
        rmse_with_padding(df['xc'].to_numpy(), df_eddy['xc'].to_numpy()),
        rmse_with_padding(df['yc'].to_numpy(), df_eddy['yc'].to_numpy()),
    )
    if rmse < rmse_min:
        rmse_min = rmse
        eddy_match = e

eddy_match


sample_eddies =     [8, 896, 2504, 2749, 382, 926, 1394, 1967]
new_sample_eddies = [5, 722, 2511, _, 1119, _, 1460, 1986]

In [ ]:
df = df_eddies[df_eddies.Eddy==eddy_match]
plt.plot(df.xc, df.yc)
plt.plot(df_eddy.xc, df_eddy.yc)
